# Model 1

In [42]:
# Import packages
import cv2
import os
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import tensorflow as tf
from tensorflow.keras import layers
from tensorflow.keras.models import Sequential
from tensorflow.keras.models import Model
import tensorflow.keras.backend as K
from tensorflow.keras import models
from tensorflow.keras.layers import Flatten, Dense, Dropout
from tensorflow.keras.optimizers import SGD
from tensorflow.keras.losses import BinaryCrossentropy
from tensorflow.keras.callbacks import LearningRateScheduler, ModelCheckpoint
from tensorflow.keras import layers
from tensorflow.keras.models import Model
from tensorflow.keras.applications import InceptionResNetV2
from tensorflow.keras import optimizers

In [43]:
hists = []

In [44]:
train_dir = "/kaggle/input/maleria/cell_images"

BATCH_SIZE = 16
IMG_SIZE = (224, 224)

train_data = tf.keras.preprocessing.image_dataset_from_directory(
    train_dir,
    label_mode="categorical",
    image_size=IMG_SIZE,
    validation_split=0.2,
    subset="training",
    batch_size=BATCH_SIZE,
    seed=42,
)

val_data = tf.keras.preprocessing.image_dataset_from_directory(
    train_dir,
    label_mode="categorical",
    image_size=IMG_SIZE,
    validation_split=0.2,
    subset="validation",
    batch_size=BATCH_SIZE,
    seed=42,
)

Found 27558 files belonging to 2 classes.
Using 22047 files for training.
Found 27558 files belonging to 2 classes.
Using 5511 files for validation.


In [45]:
# Print the class names to understand the mapping
print(train_data.class_names)

['Parasitized', 'Uninfected']


In [46]:
# Data Augmentation with Noise
data_augmentation = Sequential([
    layers.experimental.preprocessing.RandomFlip("horizontal"),
    layers.experimental.preprocessing.RandomRotation(0.2),
    layers.experimental.preprocessing.RandomZoom(0.2),
    layers.experimental.preprocessing.RandomContrast(0.2),
    layers.GaussianNoise(0.1),  # You can adjust the noise level as needed
], name ="data_augmentation")

In [47]:
base_model = tf.keras.applications.efficientnet_v2.EfficientNetV2L(include_top=False, weights='imagenet')
base_model.trainable = True

inputs = layers.Input(shape=(224, 224, 3), name="input_layer")
augmented_inputs = data_augmentation(inputs)
x = base_model(augmented_inputs, training=True)
x = layers.GlobalAveragePooling2D(name="global_average_pooling")(x)
outputs = layers.Dense(2, activation="softmax", name="output_layer")(x)
model_1 = tf.keras.Model(inputs, outputs)

In [50]:
model_1.summary()

Model: "model_2"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_layer (InputLayer)    [(None, 224, 224, 3)]     0         
                                                                 
 data_augmentation (Sequent  (None, 224, 224, 3)       0         
 ial)                                                            
                                                                 
 efficientnetv2-l (Function  (None, None, None, 1280   117746848 
 al)                         )                                   
                                                                 
 global_average_pooling (Gl  (None, 1280)              0         
 obalAveragePooling2D)                                           
                                                                 
 output_layer (Dense)        (None, 2)                 2562      
                                                           

In [51]:
model_1.compile(loss="categorical_crossentropy",
              optimizer=tf.keras.optimizers.Adam(learning_rate = 1e-4),
              metrics=["accuracy"])

In [ ]:
# checkpoint_callback = tf.keras.callbacks.ModelCheckpoint(
#     "best_bleed_model.h5",
#     save_best_only=True,
#     save_weights_only=True,
#     monitor="val_loss",
#     verbose=1
# )

# early_stopping_callback = tf.keras.callbacks.EarlyStopping(
#     monitor="val_loss",
#     patience=4,
#     verbose=1,
#     restore_best_weights=True
# )

# reduce_lr_callback = tf.keras.callbacks.ReduceLROnPlateau(
#     monitor="val_loss",
#     factor=0.2,
#     patience=2,
#     min_lr=1e-6,
#     verbose=1
# )


# callbacks = [checkpoint_callback, early_stopping_callback, reduce_lr_callback]

In [52]:
checkpoint = ModelCheckpoint('efficientnet.h5', monitor='val_loss', save_best_only=True)

In [ ]:
efficientnet = model_1.fit(train_data,
                             epochs=5,
                             validation_data=val_data,
                             batch_size=BATCH_SIZE,
                             callbacks = checkpoint)

hists.append(efficientnet)

Epoch 1/5


2024-03-15 19:52:14.802447: E tensorflow/core/grappler/optimizers/meta_optimizer.cc:961] layout failed: INVALID_ARGUMENT: Size of values 0 does not match size of permutation 4 @ fanin shape inmodel_2/efficientnetv2-l/block1b_drop/dropout/SelectV2-2-TransposeNHWCToNCHW-LayoutOptimizer


In [34]:
for hist in hists:
    print(hist)


In [ ]:
import pandas as pd

# Assuming hists is a list containing history objects
data = []
for history in hists:
    for epoch in history.epoch:
        row_data = {'epoch': epoch}
        for metric_name, metric_values in history.history.items():
            row_data[metric_name] = metric_values[epoch]
        data.append(row_data)

# Convert the data to a pandas DataFrame
df = pd.DataFrame(data)

# Save the DataFrame to a CSV file
df.to_csv('history_data.csv', index=False)


In [ ]:
plt.plot(efficientnet.history['accuracy'])
plt.plot(efficientnet.history['val_accuracy'])
plt.title(" Model_Accuracy")
plt.ylabel('Accuracy')
plt.xlabel('epoch')
plt.legend(['traning_Accuracy', 'Validation_Accuracy'])
plt.show()

In [ ]:
print(type(hists))


In [ ]:
# plt.plot(hists.history['loss'])
# plt.plot(hists.history['val_loss'])
# plt.title("Model loss")
# plt.ylabel('loss')
# plt.xlabel('epoch')
# plt.legend(['traning_loss', 'validation_loss'])
# plt.show()

# Model 2

In [ ]:
# Load MobileNetV3Large model with pretrained weights
base_model = tf.keras.applications.MobileNetV3Large(include_top=False, weights='imagenet', input_shape=(224, 224, 3))

# Freeze layers
base_model.trainable = True

# Define new top layers
flatten = Flatten()
fc1 = Dense(2048, activation='relu')
drop1 = Dropout(0.6)
fc2 = Dense(2, activation='sigmoid')

# Combine layers into a sequential model
model_2 = models.Sequential([
    base_model,
    flatten,
    fc1,
    drop1,
    fc2
])

In [ ]:
# Compile the model
model_2.compile(optimizer=SGD(learning_rate=0.001, momentum=0.9),
              loss=BinaryCrossentropy(),
              metrics=['accuracy'])

# Define learning rate scheduler
def scheduler(epoch, lr):
    if epoch % 7 == 0 and epoch != 0:
        return lr * 0.1
    else:
        return lr

lr_scheduler = LearningRateScheduler(scheduler)
checkpoint = ModelCheckpoint('MobileNet.h5', monitor='val_loss', save_best_only=True)

In [ ]:
# Print model summary
model_2.summary()

In [ ]:
# Define the number of epochs
epochs = 20

# Train the model
MobileNet = model_2.fit(train_data,  
                    epochs=epochs,
                    validation_data=val_data,  
                    callbacks=[lr_scheduler, checkpoint])
hists.append(MobileNet)

In [ ]:
for hist in hists:
    print(hist)


In [ ]:
plt.plot(MobileNet.history['loss'])
plt.plot(MobileNet.history['val_loss'])
plt.title("Model loss")
plt.ylabel('loss')
plt.xlabel('epoch')
plt.legend(['traning_loss', 'validation_loss'])
plt.show()

# Model 3

In [9]:
# Load the base model without the top layers
base_model = InceptionResNetV2(include_top=False, weights='imagenet')

# Freeze the base model
base_model.trainable = True

# Input layer
inputs = tf.keras.Input(shape=(224, 224, 3), name="input_layer")

# Augment input images
augmented_inputs = data_augmentation(inputs)

# Pass augmented inputs through the base model
x = base_model(augmented_inputs, training=True)

# Global average pooling layer
x = layers.GlobalAveragePooling2D(name="global_average_pooling")(x)

# Output layer
outputs = layers.Dense(2, activation="softmax", name="output_layer")(x)

# Define the model
model_3= tf.keras.Model(inputs, outputs)

# Display model summary
model_3.summary()


219055592/219055592 [==============================] - 1s 0us/step
Model: "model"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_layer (InputLayer)    [(None, 224, 224, 3)]     0         
                                                                 
 data_augmentation (Sequent  (None, 224, 224, 3)       0         
 ial)                                                            
                                                                 
 inception_resnet_v2 (Funct  (None, None, None, 1536   54336736  
 ional)                      )                                   
                                                                 
 global_average_pooling (Gl  (None, 1536)              0         
 obalAveragePooling2D)                                           
                                                                 
 output_layer (Dense)        (None, 2)                 3074 

In [10]:
checkpoint = ModelCheckpoint('InceptionResNet.h5', monitor='val_loss', save_best_only=True)

In [11]:
# Training
model_3.compile(optimizer=optimizers.SGD(learning_rate=0.001, momentum=0.9),
              loss='categorical_crossentropy', 
              metrics=['accuracy'])

In [20]:
print(hists)


[]


In [14]:
# Define the number of epochs
epochs = 5

# Train the model
InceptionResNet = model_3.fit(train_data, 
                    epochs=epochs,
                    validation_data=val_data,
                    callbacks = checkpoint)

hists.append(InceptionResNet)

Epoch 1/5
1378/1378 [==============================] - ETA: 0s - loss: 0.1411 - accuracy: 0.9521

/opt/conda/lib/python3.10/site-packages/keras/src/engine/training.py:3103: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(


1378/1378 [==============================] - 318s 231ms/step - loss: 0.1411 - accuracy: 0.9521 - val_loss: 0.1120 - val_accuracy: 0.9617
Epoch 2/5
1378/1378 [==============================] - 311s 225ms/step - loss: 0.1104 - accuracy: 0.9619 - val_loss: 0.0995 - val_accuracy: 0.9675
Epoch 3/5
1378/1378 [==============================] - 311s 225ms/step - loss: 0.1063 - accuracy: 0.9637 - val_loss: 0.0962 - val_accuracy: 0.9679
Epoch 4/5
1378/1378 [==============================] - 311s 226ms/step - loss: 0.0940 - accuracy: 0.9679 - val_loss: 0.0893 - val_accuracy: 0.9697
Epoch 5/5
1378/1378 [==============================] - 311s 226ms/step - loss: 0.0875 - accuracy: 0.9696 - val_loss: 0.0865 - val_accuracy: 0.9710


In [15]:
print (hists)

In [17]:
for hist in hists:
    print(hist)


In [18]:
import pandas as pd

# Assuming hists is a list containing history objects
data = []
for history in hists:
    for epoch in history.epoch:
        row_data = {'epoch': epoch}
        for metric_name, metric_values in history.history.items():
            row_data[metric_name] = metric_values[epoch]
        data.append(row_data)

# Convert the data to a pandas DataFrame
df = pd.DataFrame(data)

# Save the DataFrame to a CSV file
df.to_csv('history_data.csv', index=False)


In [16]:
plt.plot(InceptionResNetV2.history['loss'])
plt.plot(InceptionResNetV2.history['val_loss'])
plt.title("Model loss")
plt.ylabel('loss')
plt.xlabel('epoch')
plt.legend(['traning_loss', 'validation_loss'])
plt.show()

AttributeError: 'function' object has no attribute 'history'

# Model 4

## CAM & Accuracy report with random images from val_data

In [ ]:
def find_last_conv_layer(model):
    for layer in reversed(model.get_layer('efficientnetv2-l').layers):
        if isinstance(layer, tf.keras.layers.Conv2D):
            return layer
    return None

def generate_cam(model, image, class_index):
    last_conv_layer = find_last_conv_layer(model)

    if last_conv_layer is None:
        raise ValueError("No convolutional layers found in the 'efficientnetv2-l' part of the model.")


    submodel = Model(inputs=model.get_layer('efficientnetv2-l').input, outputs=last_conv_layer.output)

    preprocessed_image = tf.keras.applications.efficientnet_v2.preprocess_input(image)
    preprocessed_image = tf.expand_dims(preprocessed_image, axis=0)

    # Computing feature map from the submodel
    feature_maps = submodel(preprocessed_image)

    class_weights = model.get_layer('output_layer').get_weights()[0]


    cam = np.dot(feature_maps.numpy()[0, ..., :], class_weights[:, class_index])
    cam = (cam - np.min(cam)) / (np.max(cam) - np.min(cam))
    cam = cv2.resize(cam, (224, 224))  # Resize to match the image size

    return cam




In [ ]:
import random
random.seed(42)
val_data_iterator = iter(val_data)
bleeding_class_index = 0


bleeding_images = []


while len(bleeding_images) < 5:
    
    batch_images, batch_labels = next(val_data_iterator)
    
    
    for i in range(len(batch_labels)):
        if batch_labels[i][bleeding_class_index] == 1:
            bleeding_images.append(batch_images[i])
            if len(bleeding_images) == 5:
                break


class_index = 0


plt.figure(figsize=(15, 6))
for i, image in enumerate(bleeding_images):
    cam_image = generate_cam(model, image, class_index)

    
    plt.subplot(2, 5, i + 1)
    plt.imshow(image / 255.0)  
    plt.title(f"Image {i + 1}")

    
    plt.subplot(2, 5, i + 6)
    plt.imshow(image / 255.0)  
    plt.imshow(cam_image, cmap="jet", alpha=0.5)
    plt.title("CAM")

plt.tight_layout()
plt.show()


In [ ]:
random.seed(42)

val_data_iterator = iter(val_data)
num_images_to_plot = 20
selected_images = []
true_labels = []


for _ in range(num_images_to_plot):
    
    batch_images, batch_labels = next(val_data_iterator)
   
    random_index = random.randint(0, len(batch_images) - 1)
    selected_image = batch_images[random_index]
    selected_label = batch_labels[random_index]
    
    selected_images.append(selected_image)
    true_labels.append(selected_label)

selected_images = np.array(selected_images)

predicted_labels = model.predict(selected_images)

predicted_classes = np.argmax(predicted_labels, axis=1)

class_names = ['Parasitized', 'Uninfected']


plt.figure(figsize=(15, 12))
for i in range(num_images_to_plot):
    plt.subplot(5, 5, i + 1)
    plt.imshow(selected_images[i] / 255.0) 
    plt.title(f"Original: {class_names[np.argmax(true_labels[i])]} \nPredicted: {class_names[predicted_classes[i]]}")
    plt.axis('off')

plt.tight_layout()
plt.show()


In [ ]:
from sklearn.metrics import classification_report

y_true = []
y_pred = []

for images, labels in val_data:
    predictions = model.predict(images, verbose=0)
    y_true.extend(tf.argmax(labels, axis=1))
    y_pred.extend(tf.argmax(predictions, axis=1))

classification_metrics = classification_report(y_true, y_pred, target_names=["bleeding", "non-bleeding"])
print(classification_metrics)


In [ ]:
image_names = []
predicted_labels = []

folder_path = "/kaggle/input/wce-test/Auto-WCEBleedGen Challenge Test Dataset/Test Dataset 2"
# Create a function to load and preprocess an image
def preprocess_image(file_path):
    img = tf.io.read_file(file_path)
    img = tf.image.decode_jpeg(img, channels=3)
    img = tf.image.resize(img, (224, 224))
    img = tf.cast(img, tf.float32) / 255.0  
    return img

# Get a list of image file names in the folder and sort them
image_file_names = os.listdir(folder_path)
image_file_names.sort()

# Iterate over the images in the folder
for image_name in image_file_names:
    # Remove the file extension (.png) and save only the image name
    image_name_without_extension = os.path.splitext(image_name)[0]
    
    image_path = os.path.join(folder_path, image_name)
    
    # Preprocess the image
    image = preprocess_image(image_path)
    image = tf.expand_dims(image, axis=0)  
    predictions = model.predict(image, verbose=0)
    
    predicted_class_index = tf.argmax(predictions, axis=1).numpy()[0]
    predicted_class = "Bleeding" if predicted_class_index == 0 else "Non-Bleeding"
    
    image_names.append(image_name_without_extension)
    predicted_labels.append(predicted_class)

result_df = pd.DataFrame({"Image Name": image_names, "Predicted Label": predicted_labels})

result_df.to_csv("submission2.csv", index=False)


In [ ]:
model.save('bleed.h5')

In [ ]:
from tensorflow.keras.utils import plot_model
plot_model(model_1)